<a href="https://colab.research.google.com/github/Kealfeyne/Samsung_Bootcamp_Classic_ML/blob/main/Track1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving formula_test.csv to formula_test (2).csv
Saving formula_train.csv to formula_train (2).csv
Saving test.csv to test (2).csv
Saving train.csv to train (2).csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
dataset_train = pd.read_csv("train.csv")
dataset_formula_train = pd.read_csv("formula_train.csv")
dataset_kaggletest = pd.read_csv("test.csv")
dataset_kaggleformulatest = pd.read_csv("formula_test.csv")

In [11]:
dataset_train.head(1)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0


In [12]:
dataset_formula_train.head(1)

,H,He,Li,Be,B,C,N,O,F,Ne,...,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,29.0,Ba0.2La1.8Cu1O4


In [13]:
dataset_kaggletest.head(1)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,5,92.729214,61.051113,73.132787,37.51393,1.449309,1.146919,122.90607,35.741099,47.094633,...,2.0,2.2,1.888175,2.124829,1.557113,1.053346,2,1.114286,0.632456,0.550325


In [14]:
dataset_kaggleformulatest.head(1)

,H,He,Li,Be,B,C,N,O,F,Ne,...,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.1La1.9Ag0.5Cu0.5O4


In [15]:
dataset_train = dataset_train.drop(['critical_temp'], axis=1)

In [16]:
processed_dataset_train = pd.concat([dataset_train, dataset_formula_train], axis=1).drop(['material'], axis=1)
print(processed_dataset_train.shape)

kaggletest = pd.concat([dataset_kaggletest, dataset_kaggleformulatest], axis=1).drop(['material'], axis=1)
print(kaggletest.shape)

(17010, 168)
(4253, 167)


In [17]:
print(processed_dataset_train.isna().sum().max())
print(kaggletest.isna().sum().max())

0
0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [19]:
features_train, features_test, targets_train, targets_test = train_test_split(processed_dataset_train.drop(['critical_temp'], axis=1), processed_dataset_train['critical_temp'], test_size=0.3)
print(f'Train: {features_train.shape}, {targets_train.shape}')
print(f'Train: {features_test.shape}, {targets_test.shape}')

Train: (11907, 167), (11907,)
Train: (5103, 167), (5103,)


In [20]:
def analyze_model(model, x_train, y_train, x_test, y_test):
  model.fit(x_train, y_train)

  predictions = model.predict(x_test)
  MSE = mean_squared_error(y_test, predictions)
  return model, MSE

def get_result(model, accuracy, kaggle_features, title):
  predictions = model.predict(kaggle_features)
  result = pd.DataFrame(predictions, columns=['critical_temp']).reset_index()
  result.to_csv(f"({round(accuracy, 3)}) {title}.csv", index=False)
  files.download(f"({round(accuracy, 3)}) {title}.csv")
  return result

## Регрессия дерева решений

In [25]:
from sklearn.tree import DecisionTreeRegressor
TreeRegressor = DecisionTreeRegressor()

In [26]:
model_TreeRegressor, MSE_TreeRegressor = analyze_model(TreeRegressor, features_train, targets_train, features_test, targets_test)
MSE_TreeRegressor

162.78205631639068

In [27]:
get_result(model_TreeRegressor, MSE_TreeRegressor, kaggletest, 'TreeRegressor').shape

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(4253, 2)

## Случайный лес

In [28]:
from sklearn.ensemble import RandomForestRegressor
RandomForestRegressor = RandomForestRegressor()

In [29]:
model_RandomForestRegressor, MSE_RandomForestRegressor = analyze_model(RandomForestRegressor, features_train, targets_train, features_test, targets_test)
MSE_RandomForestRegressor

94.28532960600211

In [30]:
get_result(model_RandomForestRegressor, MSE_RandomForestRegressor, kaggletest, 'TreeRegressor').shape

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(4253, 2)

## Градиентный спуск в регрессии

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
GradientBoostingRegressor = GradientBoostingRegressor()

In [32]:
model_GradientBoostingRegressor, MSE_GradientBoostingRegressor = analyze_model(GradientBoostingRegressor, features_train, targets_train, features_test, targets_test)
MSE_GradientBoostingRegressor

156.28831153055094